### concatenate the features in the same string or not?  
chatgpt  
"If features 3, 4, and 5 are extracted from feature 1, it indicates a dependency or relationship between these features. In this case, it would be more appropriate to choose combine all the features into a single list and use a single TfidfVectorizer to transform them. By treating all the features together as a collective input, you can capture the interdependencies and relationships between them, allowing the vectorizer to consider the combined textual information from feature 1, feature 3, feature 4, and feature 5. This approach will help to capture the overall textual patterns and relationships within the data."  

"Concatenating Features: When you concatenate all the features into the same feature, you create a single unified representation that captures the information from all the features. This approach can be useful when there is a strong interdependence or correlation between the different features, and combining them into a single feature provides a more comprehensive representation of the data. By doing so, you create a higher-dimensional feature space that can potentially capture complex relationships among the features. However, this approach may also introduce noise or irrelevant information if some of the individual features are not informative for the task at hand."  

"Combining all the features might lead to the loss of feature-specific information. For example, if you have separate NER (Named Entity Recognition) tags or keywords, they might lose their distinctiveness when combined with other textual content. Scale sensitivity: If the scale or magnitude of one feature dominates the others, it might overshadow the contributions of other features during the vectorization process."  

"Each feature can be vectorized independently, allowing the vectorizer to capture specific characteristics of each feature separately."

In [17]:
from scipy.sparse import hstack
import numpy as np
import gensim
from gensim.models import Word2Vec, KeyedVectors
from nltk import word_tokenize
from scipy.sparse import hstack
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
# from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, learning_curve
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from statistics import mean
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import numpy as np
np.random.seed(42)  # Set random seed for numpy

import random
random.seed(42)  # Set random seed for random module

In [4]:
df = pd.read_csv("D:\\MS DATA SCIENCE\\NLP TESE\\data\\data_processed_selected.csv")

In [5]:
df.columns

Index(['date', 'title', 'reason', 'description', 'zone', 'narrative_tfidf',
       'narrative_embeddings', 'narrative_bert', 'narrative_tfidf_title',
       'narrative_bert_title', 'narrative_embeddings_title', 'tfidf_title',
       'title_tfidf_nolemma', 'events', 'keywords', 'embeddings_title',
       'tfidf_keywords', 'embeddings_keywords', 'events_clean',
       'results_final', 'events_tfidf', 'events_embeddings', 'orgs', 'locs',
       'entities'],
      dtype='object')

In [6]:
print(df['entities'].isna().sum())
print(df['narrative_tfidf'].isna().sum())
print(df['tfidf_title'].isna().sum())
print(df['tfidf_keywords'].isna().sum())
print(df['events_tfidf'].isna().sum())

5091
0
4709
0
23


In [7]:
def preprocess_entities(row):
    if pd.isna(row)==False:
        entities = row.lower()  # Convert to lowercase
        entities = entities.replace(',', '')  # Remove commas
        entities = ' '.join(set(entities.split()))  # Convert to set to get unique values, then join back as a string
        return entities
df['entities'] = df['entities'].apply(preprocess_entities)

In [8]:
df['entities']

0                          aboboda israel ctt
1                                        None
2                               frança roissy
3        inglés el verde lisboa corte correio
4                                        None
                         ...                 
22425                portugal reino ctt unido
22426                                    None
22427                              aliexpress
22428                                  lisboa
22429                                    None
Name: entities, Length: 22430, dtype: object

In [9]:
# Plot Learning Curves
def plot_learning_curve(model, X_train, y_train, scoring, feature_set):
    # Create StratifiedKFold cross-validator with 6 folds
    stratified_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    train_sizes, train_scores, val_scores = learning_curve(model, X_train, y_train, cv=stratified_cv, 
                                                           scoring=scoring, train_sizes=np.linspace(0.1, 1.0, 10),
                                                           random_state=42, n_jobs=-1)

    # Calculate mean and standard deviation of training and validation scores
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    valid_scores_mean = np.mean(val_scores, axis=1)
    valid_scores_std = np.std(val_scores, axis=1)

    # Plot learning curves
    plt.figure(figsize=(10, 6))
    plt.plot(train_sizes, train_scores_mean, label='Training score', color='blue')
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.2, color='blue')
    plt.plot(train_sizes, valid_scores_mean, label='Cross-validation score', color='red')
    plt.fill_between(train_sizes, valid_scores_mean - valid_scores_std,
                     valid_scores_mean + valid_scores_std, alpha=0.2, color='red')
    plt.xlabel('Number of Training Examples')
    plt.ylabel('{} Score'.format(scoring))
    plt.title('Learning Curves')
    plt.legend(loc='best')
    plt.grid()
    plt.show()

# 1. TF-IDF

In [33]:
y=df['reason']
X = df[['narrative_tfidf', 'tfidf_title', 'tfidf_keywords', 'events_tfidf', 'entities']]

In [34]:
# Split the data into a training set (70%) and a test set (30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Print the shape of each set
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (15701, 5)
y_train shape: (15701,)
X_test shape: (6729, 5)
y_test shape: (6729,)


In [44]:
def unique_words(data):
    # Split the string into individual words
    keywords_clean = data.fillna('')
    keywords_words = ' '.join(keywords_clean).split()
    # Get unique words
    unique_words = list(set(keywords_words))
    print(len(unique_words))

In [45]:
unique_words(df.narrative_tfidf)
unique_words(df.tfidf_keywords)
unique_words(df.events_tfidf)
unique_words(df.entities)
unique_words(df.tfidf_title)

25140
11339
9315
4617
2827


Generate the power set of features and evaluate different combinations to identify the most informative subset of features. 
Optimal feature combination is useful when you want to determine the combination of features that provides the best classification accuracy or predictive power for your specific task. This step will allow you to determine if incorporating additional features improves the classification performance beyond the baseline model.

In [46]:
from itertools import combinations

features = ['narrative_tfidf', 'tfidf_title', 'tfidf_keywords', 'events_tfidf', 'entities']

valid_combinations = []
for r in range(2, len(features) + 1):  # Choose r from 2 to len(features)
    for combo in combinations(features, r):
        if 'narrative_tfidf' in combo and 'tfidf_title' in combo:
            valid_combinations.append(list(combo))

# Print the valid combinations
for combo in valid_combinations:
    print(combo)

['narrative_tfidf', 'tfidf_title']
['narrative_tfidf', 'tfidf_title', 'tfidf_keywords']
['narrative_tfidf', 'tfidf_title', 'events_tfidf']
['narrative_tfidf', 'tfidf_title', 'entities']
['narrative_tfidf', 'tfidf_title', 'tfidf_keywords', 'events_tfidf']
['narrative_tfidf', 'tfidf_title', 'tfidf_keywords', 'entities']
['narrative_tfidf', 'tfidf_title', 'events_tfidf', 'entities']
['narrative_tfidf', 'tfidf_title', 'tfidf_keywords', 'events_tfidf', 'entities']


In [47]:
# rows where at least one of the columns has nan value
for t in valid_combinations:
    print(t, df[t].isna().any(axis=1).sum())

['narrative_tfidf', 'tfidf_title'] 4709
['narrative_tfidf', 'tfidf_title', 'tfidf_keywords'] 4709
['narrative_tfidf', 'tfidf_title', 'events_tfidf'] 4728
['narrative_tfidf', 'tfidf_title', 'entities'] 8649
['narrative_tfidf', 'tfidf_title', 'tfidf_keywords', 'events_tfidf'] 4728
['narrative_tfidf', 'tfidf_title', 'tfidf_keywords', 'entities'] 8649
['narrative_tfidf', 'tfidf_title', 'events_tfidf', 'entities'] 8664
['narrative_tfidf', 'tfidf_title', 'tfidf_keywords', 'events_tfidf', 'entities'] 8664


In [48]:
# rows where all the columns are NaN values
for t in valid_combinations:
    print(t,df[t].isna().all(axis=1).sum())

['narrative_tfidf', 'tfidf_title'] 0
['narrative_tfidf', 'tfidf_title', 'tfidf_keywords'] 0
['narrative_tfidf', 'tfidf_title', 'events_tfidf'] 0
['narrative_tfidf', 'tfidf_title', 'entities'] 0
['narrative_tfidf', 'tfidf_title', 'tfidf_keywords', 'events_tfidf'] 0
['narrative_tfidf', 'tfidf_title', 'tfidf_keywords', 'entities'] 0
['narrative_tfidf', 'tfidf_title', 'events_tfidf', 'entities'] 0
['narrative_tfidf', 'tfidf_title', 'tfidf_keywords', 'events_tfidf', 'entities'] 0


### combine everything in a string and fillana('')
use all the features in a string and use combinations where at least one of the features exists for that row, 
- eg.x=['narrative_tfidf', 'tfidf_title', 'tfidf_keywords', 'events_tfidf', 'entities'] if a row doesnt have a value for column events_tfidf and entities, for feature_combination x we only use 'narrative_tfidf', 'tfidf_title', 'tfidf_keywords'. If a row has values for all columns in x we use 'narrative_tfidf', 'tfidf_title', 'tfidf_keywords', 'events_tfidf', 'entities'.
- ['tfidf_title', 'entities'] has 1151 rows with no values for either column, so I didnt use this feature combination.

In [49]:
def applyclf(X_train, y_train, vectorizer, model, feature_set):
    X_train_selected = X_train[feature_set].apply(lambda x: ' '.join(x.fillna('').astype(str)), axis=1)
    
    pipeline = Pipeline([
        ('vectorizer', vectorizer),
        ('classifier', model)
    ])

    # Define cross-validation strategy (e.g., StratifiedKFold with 5 folds)
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    # Lists to store predictions and true labels
    all_true_labels = []
    all_predicted_labels = []

    # Perform cross-validation and accumulate the confusion matrix
    for train_index, val_index in kf.split(X_train_selected, y_train):
        X_train_fold, X_val_fold = X_train_selected.iloc[train_index], X_train_selected.iloc[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

        # Fit the model on the training fold
        pipeline.fit(X_train_fold, y_train_fold)

        # Predict on the validation fold
        y_pred_val = pipeline.predict(X_val_fold)
        
        # Append true labels and predicted labels
        all_true_labels.extend(y_val_fold)
        all_predicted_labels.extend(y_pred_val)
        
    # Get the number of features from the vectorizer
    num_features = len(vectorizer.get_feature_names_out())
    # Print the number of features, confusion matrix, and classification report
    print(f"Number of Features: {num_features}")
            
    # Generate an overall confusion matrix
    confusion_mat = confusion_matrix(all_true_labels, all_predicted_labels, labels=[0,1,2,3])
    
    # Calculate the classification report
    class_report = classification_report(all_true_labels, all_predicted_labels)

    # Print the confusion matrix and classification report
    print("Confusion Matrix:")
    print(confusion_mat)

    print("\nClassification Report:")
    print(class_report)

### XGBOOST

In [51]:
config= {'ngram_range': (1,2), 'max_df':0.80, 'min_df': 0.01}  
for combo in valid_combinations:
    print('Starting combination: ', combo)
    applyclf(X_train, y_train, TfidfVectorizer(**config), XGBClassifier(random_state=42), combo)

Starting combination:  ['narrative_tfidf', 'tfidf_title']
Number of Features: 1240
Confusion Matrix:
[[3554  130 1774  118]
 [ 955  110  427   61]
 [1195   52 6226   37]
 [ 588   35  219  220]]

Classification Report:
              precision    recall  f1-score   support

           0       0.56      0.64      0.60      5576
           1       0.34      0.07      0.12      1553
           2       0.72      0.83      0.77      7510
           3       0.50      0.21      0.29      1062

    accuracy                           0.64     15701
   macro avg       0.53      0.44      0.45     15701
weighted avg       0.61      0.64      0.61     15701

Starting combination:  ['narrative_tfidf', 'tfidf_title', 'tfidf_keywords']
Number of Features: 1271
Confusion Matrix:
[[3505  135 1797  139]
 [ 930  119  451   53]
 [1193   35 6250   32]
 [ 612   26  215  209]]

Classification Report:
              precision    recall  f1-score   support

           0       0.56      0.63      0.59      5576
  

### LINEARSVC

In [52]:
config= {'ngram_range': (1,2), 'max_df':0.80, 'min_df': 0.01}  
for combo in valid_combinations:
    print('Starting combination: ', combo)
    applyclf(X_train, y_train, TfidfVectorizer(**config), LinearSVC(), combo)

Starting combination:  ['narrative_tfidf', 'tfidf_title']
Number of Features: 1240
Confusion Matrix:
[[3394  180 1810  192]
 [ 882  126  459   86]
 [1157   53 6232   68]
 [ 507   56  244  255]]

Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.61      0.59      5576
           1       0.30      0.08      0.13      1553
           2       0.71      0.83      0.77      7510
           3       0.42      0.24      0.31      1062

    accuracy                           0.64     15701
   macro avg       0.50      0.44      0.45     15701
weighted avg       0.60      0.64      0.61     15701

Starting combination:  ['narrative_tfidf', 'tfidf_title', 'tfidf_keywords']
Number of Features: 1271
Confusion Matrix:
[[3312  179 1912  173]
 [ 877  127  475   74]
 [1138   53 6247   72]
 [ 502   46  252  262]]

Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.59      0.58      5576
  

# 2. EMBEDDINGS

### combine all features in the same string  and fillna('')
use feature combinations where at least one feature exists

In [10]:
y=df['reason']
X = df[['narrative_embeddings', 'embeddings_title', 'embeddings_keywords', 'events_embeddings', 'entities']]

In [11]:
# Split the data into a training set (70%) and a test set (30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Print the shape of each set
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (15701, 5)
y_train shape: (15701,)
X_test shape: (6729, 5)
y_test shape: (6729,)


"Calculate average word embeddings for each column:  
For each column, follow these steps:  

a. Iterate over each row in the column.    
b. Tokenize the text in the current row into individual words.
c. For each word, check if it exists in the word embeddings model. If so, retrieve its word embedding vector.  
d. If any word embeddings are found for the row, calculate the average of all the word embeddings for that row.  
e. Append the average word embedding vector to a list that represents the average word embeddings for the respective column.  

Repeat this process for each column, resulting in 5 lists.  
Combine/concatenate average word embeddings."

In [12]:
def embeddings_transformation(X, model):
    '''returns the mean of the embeddings of the document X'''
    # initiating a sentence with all zeros
    embedding_size = 600  
    X_transformed = np.zeros((len(X), embedding_size))
   
    # Loop over each string in X
    for i, sentence in enumerate(X):
        # Loop over each word in the sentence and, if it is in the model's vocabulary, add its feature vector to the total
        embeddings = [model[word] for word in sentence.split() if word in model]
        if embeddings:
            X_transformed[i] = np.mean(embeddings, axis=0)
    return X_transformed

In [13]:
from itertools import combinations

features = ['narrative_embeddings', 'embeddings_title', 'embeddings_keywords', 'events_embeddings', 'entities']

valid_combinations = []
for r in range(2, len(features) + 1):  # Choose r from 2 to len(features)
    for combo in combinations(features, r):
        if 'narrative_embeddings' in combo and 'embeddings_title' in combo:
            valid_combinations.append(list(combo))

# Print the valid combinations
for combo in valid_combinations:
    print(combo)

['narrative_embeddings', 'embeddings_title']
['narrative_embeddings', 'embeddings_title', 'embeddings_keywords']
['narrative_embeddings', 'embeddings_title', 'events_embeddings']
['narrative_embeddings', 'embeddings_title', 'entities']
['narrative_embeddings', 'embeddings_title', 'embeddings_keywords', 'events_embeddings']
['narrative_embeddings', 'embeddings_title', 'embeddings_keywords', 'entities']
['narrative_embeddings', 'embeddings_title', 'events_embeddings', 'entities']
['narrative_embeddings', 'embeddings_title', 'embeddings_keywords', 'events_embeddings', 'entities']


In [14]:
def applyclf_emb(X_train, y_train, model, feature_set, emb_model):
    X_train_selected = X_train[feature_set].apply(lambda x: ' '.join(x.fillna('').astype(str)), axis=1)
    # Transform the text data into embeddings
    X_train_embeddings = embeddings_transformation(X_train_selected, emb_model)
    
    # Create a CountVectorizer and Multinomial Naive Bayes pipeline
    pipeline = Pipeline([
        ('classifier', model)
    ])

    # Define cross-validation strategy (e.g., StratifiedKFold with 5 folds)
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    # Lists to store predictions and true labels
    all_true_labels = []
    all_predicted_labels = []

    # Perform cross-validation and accumulate the confusion matrix
    for train_index, val_index in kf.split(X_train_embeddings, y_train):
        X_train_fold, X_val_fold = X_train_embeddings[train_index], X_train_embeddings[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

        # Fit the model on the training fold
        pipeline.fit(X_train_fold, y_train_fold)

        # Predict on the validation fold
        y_pred_val = pipeline.predict(X_val_fold)
        
        # Append true labels and predicted labels
        all_true_labels.extend(y_val_fold)
        all_predicted_labels.extend(y_pred_val)
        
    # Generate an overall confusion matrix
    confusion_mat = confusion_matrix(all_true_labels, all_predicted_labels, labels=[0,1,2,3])
    
    # Calculate the classification report
    class_report = classification_report(all_true_labels, all_predicted_labels)

    # Print the confusion matrix and classification report
    print("Confusion Matrix:")
    print(confusion_mat)

    print("\nClassification Report:")
    print(class_report)

## 2.1. WORD2VEC

In [15]:
w2v= KeyedVectors.load_word2vec_format('D:\\MS DATA SCIENCE\\NLP TESE\\embeddings\\skip_s600_word2vec.txt')

### XGBOOST

In [29]:
for combo in valid_combinations:
    print('Starting combination: ', combo)
    applyclf_emb(X_train, y_train, XGBClassifier(random_state=42), combo, w2v)

Starting combination:  ['narrative_embeddings', 'embeddings_title']
Confusion Matrix:
[[3557   98 1847   74]
 [ 952   80  475   46]
 [1183   29 6276   22]
 [ 621   28  267  146]]

Classification Report:
              precision    recall  f1-score   support

           0       0.56      0.64      0.60      5576
           1       0.34      0.05      0.09      1553
           2       0.71      0.84      0.77      7510
           3       0.51      0.14      0.22      1062

    accuracy                           0.64     15701
   macro avg       0.53      0.42      0.42     15701
weighted avg       0.61      0.64      0.60     15701

Starting combination:  ['narrative_embeddings', 'embeddings_title', 'embeddings_keywords']
Confusion Matrix:
[[3509   76 1919   72]
 [ 938   66  510   39]
 [1222   15 6252   21]
 [ 599   43  291  129]]

Classification Report:
              precision    recall  f1-score   support

           0       0.56      0.63      0.59      5576
           1       0.33    

### GRADIENT BOOSTING

In [19]:
for combo in valid_combinations:
    print('Starting combination: ', combo)
    applyclf_emb(X_train, y_train, GradientBoostingClassifier(random_state=42), combo, w2v)

Starting combination:  ['narrative_embeddings', 'embeddings_title']
Confusion Matrix:
[[3550   66 1846  114]
 [ 973   56  465   59]
 [1183   27 6257   43]
 [ 609   30  247  176]]

Classification Report:
              precision    recall  f1-score   support

           0       0.56      0.64      0.60      5576
           1       0.31      0.04      0.06      1553
           2       0.71      0.83      0.77      7510
           3       0.45      0.17      0.24      1062

    accuracy                           0.64     15701
   macro avg       0.51      0.42      0.42     15701
weighted avg       0.60      0.64      0.60     15701

Starting combination:  ['narrative_embeddings', 'embeddings_title', 'embeddings_keywords']
Confusion Matrix:
[[3449   69 1950  108]
 [ 949   54  494   56]
 [1229   19 6226   36]
 [ 597   25  292  148]]

Classification Report:
              precision    recall  f1-score   support

           0       0.55      0.62      0.58      5576
           1       0.32    

## 2.2 GLOVE

In [40]:
glove= KeyedVectors.load_word2vec_format('D:\\MS DATA SCIENCE\\NLP TESE\\embeddings\\glove_s600.txt')

Feature Combination: Once you have encoded the text features using word embeddings, you can combine them in different ways to create new feature representations. 
1. Concatenation: Concatenate the word embeddings of different features together to create a single long vector. For instance, if 'narrative_tfidf' and 'tfidf_title' are both word embeddings of size 300, you can concatenate them to obtain a combined feature vector of size 600.

2. Averaging: Take the average of the word embeddings of different features. This is useful when you want to retain the overall information from each feature but reduce the dimensionality. For example, if 'narrative_tfidf' and 'tfidf_title' are both word embeddings of size 300, you can average them element-wise to obtain a combined feature vector of size 300.

3. Element-wise operations: Perform element-wise operations like addition or multiplication on the word embeddings of different features. This can capture interactions between the features at a more granular level. For example, you can add or multiply the word embeddings of 'tfidf_title' and 'entities' element-wise to obtain a combined feature vector.

Since the texts of each feature are long, if we find the word embeddings of different features and append or concatenate them together, that would result in a longer feature vector. For example, if you we two word embeddings of size 300, concatenating them would result in a combined feature vector of size 600.  

With averagging, the word embeddings of different features are element-wise averaged together. This means that the corresponding elements of the word embeddings are summed up and divided by the number of features. For example, if you we two word embeddings of size 300, averaging them would result in a combined feature vector of size 300. The averaging operation reduces the dimensionality of the feature vector but retains the overall information from each feature.

So since we have 5 features of long texts in separate columns, we are going to perform averaging.  
Calculate the average of the word embeddings for each feature individually. Then, can concatenate the averaged feature vectors to create a single combined feature vector. If each feature_avg has a dimension of 600, and we have 5 features, the dimension of the combined_feature_vector would be 5 * 600 = 3000.

### XGBOOST

In [32]:
for combo in valid_combinations:
    print('Starting combination: ', combo)
    applyclf_emb(X_train, y_train, XGBClassifier(random_state=42), combo, glove)

Starting combination:  ['narrative_embeddings', 'embeddings_title']
Confusion Matrix:
[[3578   77 1862   59]
 [ 965   65  479   44]
 [1232   29 6228   21]
 [ 659   30  256  117]]

Classification Report:
              precision    recall  f1-score   support

           0       0.56      0.64      0.60      5576
           1       0.32      0.04      0.07      1553
           2       0.71      0.83      0.76      7510
           3       0.49      0.11      0.18      1062

    accuracy                           0.64     15701
   macro avg       0.52      0.41      0.40     15701
weighted avg       0.60      0.64      0.60     15701

Starting combination:  ['narrative_embeddings', 'embeddings_title', 'embeddings_keywords']
Confusion Matrix:
[[3436   82 1991   67]
 [ 968   67  491   27]
 [1252   20 6224   14]
 [ 630   27  292  113]]

Classification Report:
              precision    recall  f1-score   support

           0       0.55      0.62      0.58      5576
           1       0.34    

### LINEARSVC

In [33]:
for combo in valid_combinations:
    print('Starting combination: ', combo)
    applyclf_emb(X_train, y_train, LinearSVC(), combo, glove)

Starting combination:  ['narrative_embeddings', 'embeddings_title']
Confusion Matrix:
[[3647   36 1792  101]
 [ 976   49  462   66]
 [1119   13 6352   26]
 [ 642   11  231  178]]

Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.65      0.61      5576
           1       0.45      0.03      0.06      1553
           2       0.72      0.85      0.78      7510
           3       0.48      0.17      0.25      1062

    accuracy                           0.65     15701
   macro avg       0.55      0.42      0.42     15701
weighted avg       0.62      0.65      0.61     15701

Starting combination:  ['narrative_embeddings', 'embeddings_title', 'embeddings_keywords']
Confusion Matrix:
[[3555   48 1868  105]
 [ 971   51  471   60]
 [1119   16 6346   29]
 [ 611   15  274  162]]

Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.64      0.60      5576
           1       0.39    